In [1]:
from os.path import join
from os import walk
import pandas as pd
import healpy as hp
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from tqdm.notebook import tqdm
import threading

In [2]:
def radec2pix(ra, dec, nside):
    c = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
    
    l = c.galactic.l.degree
    b = c.galactic.b.degree
    
    return hp.ang2pix(nside, l, b, nest=True, lonlat=True)

In [12]:
def selected_to_healpy(origdir, filename, findir, nside):
    
    table = pd.read_csv(join(origdir, filename), index_col='index')
    table.index = np.arange(table.shape[0])
    table['npix'] = np.NaN

    ra = np.array(table['RA'])
    dec = np.array(table['DEC'])
    table['npix'] = radec2pix(ra, dec, nside)
    table.index.name='index'
    table.to_csv(join(findir, 'nside_' + str(nside) + filename))

In [13]:
selected_to_healpy('/home/rt2122/Data/selected_north/', 'sweep-150p050-160p055.csv', 
                   '/home/rt2122/Data/north_pix/', 512)

In [14]:
t = pd.read_csv('/home/rt2122/Data/north_pix/nside_512sweep-150p050-160p055.csv', 
                index_col='index')
t.head()

,BRICKID,RA,DEC,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,npix
index,,,,,,,,,,,
0,601061,155.154441,54.627220,0.437298,0.833180,1.110131,-17.775790,5.131460,-0.951051,-67.833530,385429
1,601061,155.179560,54.625992,0.086432,0.517316,1.160431,2.708611,1.031565,-40.575610,-431.941220,385429
2,601061,154.996849,54.626928,38.270237,68.789970,106.425580,79.402970,52.157444,236.747150,21.125671,385343
3,601061,155.000979,54.628108,0.774508,2.091712,8.493885,29.833971,13.815964,60.465540,769.855100,385343
4,601061,155.003931,54.627518,0.566023,1.041511,2.184387,11.005062,11.851215,42.392258,-307.422760,385343


In [16]:
orig_dir = '/home/rt2122/Data/selected_north/'
files = next(walk(orig_dir))[-1]
fin_dir = '/home/rt2122/Data/north_pix/'
files

['sweep-150p060-160p065.csv',
 'sweep-160p055-170p060.csv',
 'sweep-160p060-170p065.csv',
 'sweep-150p055-160p060.csv',
 'sweep-150p050-160p055.csv',
 'sweep-160p050-170p055.csv']

In [17]:
for file in files:
    selected_to_healpy(orig_dir, file, fin_dir, 512)

/home/rt2122/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
